In [1]:
import numpy as np
import json
import torch
import torch.nn as nn
from networks import SCLIPNN
import clip
from multilingual_clip import pt_multilingual_clip
import transformers
from PIL import Image
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import torchvision.transforms.functional as fn
import pandas as pd
import yaml
import os
from datetime import datetime
from experiment import *

In [2]:
#######################################
# Models functions
#######################################
def get_sbert_and_clip_and_mclip_models():
    sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    print("SBERT model loaded")
    clip_model, preprocess = clip.load("ViT-B/32", device=device)
    print("CLIP model loaded")
    mclip_model = pt_multilingual_clip.MultilingualCLIP.from_pretrained('M-CLIP/XLM-Roberta-Large-Vit-B-32')
    mclip_tokenizer = transformers.AutoTokenizer.from_pretrained('M-CLIP/XLM-Roberta-Large-Vit-B-32')
    print("MCLIP model loaded")
    return sbert_model.eval(), (clip_model.eval(), preprocess), (mclip_model.eval(), mclip_tokenizer)

def get_mclip_embeddings(sentences, mclip_model, mclip_tokenizer):
    with torch.no_grad():
        mclip_embeddings = mclip_model.forward(sentences, mclip_tokenizer).to(device)
    return mclip_embeddings

In [3]:
#######################################
# Additional Functions
#######################################
def get_results_three(sbert_lang_performance, clip_lang_performance, mclip_per,
                      sbert_lang_mrr, clip_lang_mrr, mclip_mrr,
                      sbert_lang_errors, clip_lang_errors, mclip_error,
                      sbert_acc, clip_acc, mclip_acc ):
    results = pd.DataFrame({"SBERT":sbert_lang_performance, "CLIP": clip_lang_performance, "MCLIP":mclip_per,
                            "MRR sbert":sbert_lang_mrr, "MRR clip": clip_lang_mrr, "MRR mclip": mclip_mrr,
                            "error SBERT":sbert_lang_errors, "error CLIP":clip_lang_errors, "error MCLIP":mclip_error,
                            "SBERT Accuracy":sbert_acc, "CLIP Accuracy": clip_acc, "MCLIP Accuracy":mclip_acc
                       },
                       index=languages)
    return results

def show_plot(languages, sbert_lang, clip_lang, mclip_lang, metric='MRR'):
    X_axis = np.arange(len(languages.keys()))
    figure_name = plt.figure(figsize=(20, 8))
    plt.bar(X_axis-0.2, sbert_lang, 0.2, color = 'blue', edgecolor = 'black', capsize=7, label='SCLIP '+metric)
    plt.bar(X_axis+0.0, clip_lang, 0.2, color = 'red', edgecolor = 'black', capsize=7, label='CLIP '+metric)
    plt.bar(X_axis+0.2, mclip_lang, 0.2, color = 'olive', edgecolor = 'black', capsize=7, label='MCLIP '+metric)
    plt.xticks(rotation = 45)
    plt.xticks(X_axis, languages.keys())
    plt.tick_params(axis='both', which='major', labelsize=20)
    plt.legend(fontsize=15)
    plt.grid()
    plt.show()
    
def reciprocal_rank(probs, value):
    N = len(probs)
    copy_probs = list(probs.copy())
    for i in range(N):
        max_value = max(copy_probs)
        if max_value == value:
            return 1/(i + 1)
        else:
            copy_probs.remove(max_value)
    return 1/N

def get_MRR(languages, name_of_model, sbert_model, clip_model, mclip_model, mclip_tokenizer, captions, images_features):
    sbert_lang_performance = []
    clip_lang_performance = []
    mclip_lang_performance = []
    sbert_lang_errors = []
    clip_lang_errors = []
    mclip_lang_errors = []
    sbert_lang_mrr = []
    clip_lang_mrr = []
    mclip_lang_mrr = []
    s_acc = []
    c_acc = []
    m_acc = []
    vetoed = []
    for lang, code in languages.items():
        print("Processing captions in "+ lang +"...")

        with torch.no_grad():
            torch_features = get_sbert_embeddings(captions[lang],sbert_model) 
            try:
                sbert_features = sbert_to_clip(torch_features,name_of_model).type(torch.float16)
                print("SBERT features ready. Timestamp: {}".format(datetime.now()))
            except:
                print("[SBERT] Not able to extract features in {}. Skipping language {}".format(lang, code))
                vetoed.append(lang)
                continue
            try:
                clip_features = get_clip_embeddings(captions[lang],clip_model).to(device)
                print("CLIP features ready. Timestamp: {}".format(datetime.now()))
            except:
                print("[CLIP] Not able to tokenize in {}. Skipping language {}".format(lang, code))
                vetoed.append(lang)
                continue
            #try
            mclip_features = get_mclip_embeddings(captions[lang],mclip_model,mclip_tokenizer)
            print("MCLIP features ready. Timestamp: {}".format(datetime.now()))
            #except:
                #print("[MCLIP] Not able to extract features in {}. Skipping language {}".format(lang, code))
                #vetoed.append(lang)
                #continue
            
        print("Encodings complete")

        sbert_performance = []
        clip_performance = []
        mclip_performance = []
        sbert_errors = 0
        clip_errors = 0
        mclip_errors = 0
        sbert_rr = 0
        clip_rr = 0
        mclip_rr = 0
        counter = 0
        s_correct = 0
        c_correct = 0
        m_correct = 0
        
        for image_feature in images_features:
            # Get the probabilities for SBERT and CLIP
            logits_image_sbert, logits_text_sbert = get_logits(image_feature, sbert_features)#,'sbert')
            logits_image_mclip, logits_text_mclip = get_logits(image_feature, mclip_features)#,'mclip')
            logits_image_clip, logits_text_clip = get_logits(image_feature, clip_features)#,'clip')
            probs_sbert = logits_image_sbert.softmax(dim=-1).cpu().detach().numpy()
            probs_mclip = logits_image_mclip.softmax(dim=-1).cpu().detach().numpy()
            probs_clip = logits_image_clip.softmax(dim=-1).cpu().detach().numpy()
            
            # Append the probs to array            
            ps = probs_sbert[counter]
            sbert_rr += reciprocal_rank(probs_sbert,ps)
            sbert_performance.append(ps)
            if ps < max(probs_sbert):
                sbert_errors += 1
            elif ps == max(probs_sbert):
                s_correct += 1
            pc = probs_clip[counter]
            clip_rr += reciprocal_rank(probs_clip,pc)
            clip_performance.append(pc)
            if pc < max(probs_clip):
                clip_errors += 1
            elif pc == max(probs_clip):
                c_correct += 1
            pm = probs_mclip[counter]
            mclip_rr += reciprocal_rank(probs_mclip,pm)
            mclip_performance.append(pm)
            if pm < max(probs_mclip):
                mclip_errors += 1
            elif pm == max(probs_mclip):
                m_correct += 1
            counter += 1

            #if counter % 100 == 0:
            #    print("{} images already processed for {}".format(counter,lang))

        print('-'*70)
        # print("Images processed: {}".format(counter))
        # print("Classifications errors: SBERT --> {} ; CLIP --> {}".format(sbert_errors,clip_errors))
        sbert_lang_performance.append(round(sum(sbert_performance)/counter,6))
        clip_lang_performance.append(round(sum(clip_performance)/counter,6))
        mclip_lang_performance.append(round(sum(mclip_performance)/counter,6))
        sbert_lang_mrr.append(round(sbert_rr/counter,4))
        clip_lang_mrr.append(round(clip_rr/counter,4))
        mclip_lang_mrr.append(round(mclip_rr/counter,4))
        sbert_lang_errors.append(round(sbert_errors/counter,4))
        clip_lang_errors.append(round(clip_errors/counter,4))
        mclip_lang_errors.append(round(mclip_errors/counter,4))
        s_acc.append(round(s_correct/counter,4))
        c_acc.append(round(c_correct/counter,4))
        m_acc.append(round(m_correct/counter,4))
    print("Done")
    print("Forbidden Languages: {}".format(vetoed))
    
    return sbert_lang_performance, clip_lang_performance, mclip_lang_performance, sbert_lang_mrr, clip_lang_mrr, mclip_lang_mrr, sbert_lang_errors, clip_lang_errors, mclip_lang_errors, s_acc, c_acc, m_acc

In [4]:
def run_experiment(name_of_model, dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    sbert_model, (clip_model, preprocess), (mclip_model, mclip_tokenizer) = get_sbert_and_clip_and_mclip_models()
    with open(os.path.join("preprocessing", "config.yml"), "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    directory = cfg[dataset]["out_dir"]
    image_directory = cfg[dataset]["image_dir"]
    print("Image_directory: {}".format(image_directory))
    languages = cfg["languages"]
    model_dir = cfg["models"]["model_dir"]
    name_of_model = name_of_model #'coco_NN_1000_e300_s400000.pt'
    trained_model = os.path.join(model_dir,name_of_model)
    images, captions = get_images_and_captions(directory,languages)
    images_features = get_image_features(images["english"], image_directory, clip_model, preprocess)
    sbert_per, clip_per, mclip_per, sbert_MRR, clip_MRR, mclip_MRR, sbert_errors, clip_errors, mclip_errors, s_acc, c_acc, m_acc = get_MRR(languages, name_of_model, sbert_model, clip_model, mclip_model, mclip_tokenizer, captions, images_features)
    results = get_results_three(sbert_per,clip_per,mclip_per,sbert_MRR,clip_MRR,mclip_MRR,sbert_errors, clip_errors,mclip_errors,s_acc,c_acc,m_acc)
    metrics = [sbert_per, clip_per, mclip_per, sbert_MRR, clip_MRR, mclip_MRR, sbert_errors, clip_errors, mclip_errors, s_acc, c_acc, m_acc]
    return metrics, results

In [27]:
models = ['coco_NN_1000_e300_s400000.pt', 'gcc_NN_750_e300_s2654581.pt', 'gco_NN3_1000_e300_s3127982.pt']

In [33]:
model = 'coco_NN_1000_e300_s400000.pt'
metrics_coco, results = run_experiment(model,'coco')
results.to_csv("coco_results.csv")
results

SBERT model loaded
CLIP model loaded


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/coco/images/val2017
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 10:06:35.394531
CLIP features ready. Timestamp: 2023-04-28 10:06:35.617109
MCLIP features ready. Timestamp: 2023-04-28 10:07:00.887461
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 10:07:01.485948
CLIP features ready. Timestamp: 2023-04-28 10:07:01.723898
MCLIP features ready. Timestamp: 2023-04-28 10:07:27.132200
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 10:07:27.809254
CLIP features ready. Timestamp: 2023-04-28 10:07:28.059619
MCLIP features ready. Timestamp: 2023-04-28 10:07:59.370912
Encodings complete
----------------------------------------------------------------------
Processing cap

,SBERT,CLIP,MCLIP,MRR sbert,MRR clip,MRR mclip,error SBERT,error CLIP,error MCLIP,SBERT Accuracy,CLIP Accuracy,MCLIP Accuracy
english,0.010966,0.015540,0.015782,0.5255,0.6271,0.6211,0.6022,0.4844,0.5022,0.3978,0.5156,0.4978
spanish,0.010154,0.006589,0.015533,0.5026,0.3865,0.6123,0.6289,0.7400,0.5111,0.3711,0.2600,0.4889
italian,0.009808,0.005727,0.015199,0.4705,0.3156,0.6074,0.6622,0.7956,0.5133,0.3378,0.2044,0.4867
german,0.010281,0.005582,0.015246,0.4780,0.3107,0.6160,0.6600,0.8044,0.5067,0.3400,0.1956,0.4933
french,0.010296,0.007073,0.015602,0.4762,0.3922,0.6248,0.6600,0.7222,0.4889,0.3400,0.2778,0.5111
polish,0.008881,0.002939,0.015068,0.4450,0.1087,0.6129,0.6844,0.9422,0.5044,0.3156,0.0578,0.4956


In [36]:
model = 'gcc_NN_750_e300_s2654581.pt'
metrics_gcc, results = run_experiment(model,'coco')
results.to_csv("gcc_results.csv")
results

SBERT model loaded
CLIP model loaded


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/coco/images/val2017
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 10:18:45.477952
CLIP features ready. Timestamp: 2023-04-28 10:18:45.711112
MCLIP features ready. Timestamp: 2023-04-28 10:19:10.907171
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 10:19:11.497721
CLIP features ready. Timestamp: 2023-04-28 10:19:11.779344
MCLIP features ready. Timestamp: 2023-04-28 10:19:37.011449
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 10:19:37.652666
CLIP features ready. Timestamp: 2023-04-28 10:19:37.884281
MCLIP features ready. Timestamp: 2023-04-28 10:20:09.013270
Encodings complete
----------------------------------------------------------------------
Processing cap

,SBERT,CLIP,MCLIP,MRR sbert,MRR clip,MRR mclip,error SBERT,error CLIP,error MCLIP,SBERT Accuracy,CLIP Accuracy,MCLIP Accuracy
english,0.005126,0.015540,0.015782,0.3632,0.6271,0.6211,0.7533,0.4844,0.5022,0.2467,0.5156,0.4978
spanish,0.005029,0.006589,0.015533,0.3417,0.3865,0.6123,0.7844,0.7400,0.5111,0.2156,0.2600,0.4889
italian,0.004846,0.005727,0.015199,0.3267,0.3156,0.6074,0.7978,0.7956,0.5133,0.2022,0.2044,0.4867
german,0.005038,0.005582,0.015246,0.3304,0.3107,0.6160,0.7933,0.8044,0.5067,0.2067,0.1956,0.4933
french,0.005118,0.007073,0.015602,0.3299,0.3922,0.6248,0.8067,0.7222,0.4889,0.1933,0.2778,0.5111
polish,0.004615,0.002939,0.015068,0.3094,0.1087,0.6129,0.8067,0.9422,0.5044,0.1933,0.0578,0.4956


In [37]:
model = 'gco_NN3_1000_e300_s3127982.pt'
metrics_gco, results = run_experiment(model,'coco')
results.to_csv("gco_results.csv")
results

SBERT model loaded
CLIP model loaded


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/coco/images/val2017
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 10:36:49.963093
CLIP features ready. Timestamp: 2023-04-28 10:36:50.261039
MCLIP features ready. Timestamp: 2023-04-28 10:37:15.573270
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 10:37:16.168520
CLIP features ready. Timestamp: 2023-04-28 10:37:16.457189
MCLIP features ready. Timestamp: 2023-04-28 10:37:41.800632
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 10:37:42.484480
CLIP features ready. Timestamp: 2023-04-28 10:37:42.767938
MCLIP features ready. Timestamp: 2023-04-28 10:38:13.787704
Encodings complete
----------------------------------------------------------------------
Processing cap

,SBERT,CLIP,MCLIP,MRR sbert,MRR clip,MRR mclip,error SBERT,error CLIP,error MCLIP,SBERT Accuracy,CLIP Accuracy,MCLIP Accuracy
english,0.006332,0.015540,0.015782,0.3891,0.6271,0.6211,0.7400,0.4844,0.5022,0.2600,0.5156,0.4978
spanish,0.006130,0.006589,0.015533,0.3665,0.3865,0.6123,0.7644,0.7400,0.5111,0.2356,0.2600,0.4889
italian,0.005904,0.005727,0.015199,0.3454,0.3156,0.6074,0.7822,0.7956,0.5133,0.2178,0.2044,0.4867
german,0.006207,0.005582,0.015246,0.3709,0.3107,0.6160,0.7489,0.8044,0.5067,0.2511,0.1956,0.4933
french,0.006253,0.007073,0.015602,0.3610,0.3922,0.6248,0.7711,0.7222,0.4889,0.2289,0.2778,0.5111
polish,0.005637,0.002939,0.015068,0.3447,0.1087,0.6129,0.7689,0.9422,0.5044,0.2311,0.0578,0.4956


In [5]:
model = 'coco_NN_1000_e300_s400000.pt'
metrics_cross_coco, results = run_experiment(model,'crossmodal')
results.to_csv("coco_cross_results.csv")
model = 'gcc_NN_750_e300_s2654581.pt'
metrics_cross_gcc, results = run_experiment(model,'crossmodal')
results.to_csv("gcc_cross_results.csv")
model = 'gco_NN3_1000_e300_s3127982.pt'
metrics_cross_gco, results = run_experiment(model,'crossmodal')
results.to_csv("gco_cross_results.csv")

SBERT model loaded
CLIP model loaded


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/crossmodal/images
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 11:15:44.743217
CLIP features ready. Timestamp: 2023-04-28 11:15:45.083221
MCLIP features ready. Timestamp: 2023-04-28 11:16:10.920723
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 11:16:11.543665
CLIP features ready. Timestamp: 2023-04-28 11:16:11.856623
MCLIP features ready. Timestamp: 2023-04-28 11:16:36.711581
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 11:16:37.480036
CLIP features ready. Timestamp: 2023-04-28 11:16:37.759243
MCLIP features ready. Timestamp: 2023-04-28 11:17:08.851150
Encodings complete
----------------------------------------------------------------------
Processing capti

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/crossmodal/images
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 11:18:28.409915
CLIP features ready. Timestamp: 2023-04-28 11:18:28.734859
MCLIP features ready. Timestamp: 2023-04-28 11:18:53.478510
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 11:18:54.158056
CLIP features ready. Timestamp: 2023-04-28 11:18:54.442986
MCLIP features ready. Timestamp: 2023-04-28 11:19:18.517133
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 11:19:19.218101
CLIP features ready. Timestamp: 2023-04-28 11:19:19.475224
MCLIP features ready. Timestamp: 2023-04-28 11:19:49.239127
Encodings complete
----------------------------------------------------------------------
Processing capti

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MCLIP model loaded
Image_directory: /home/acordova/SCLIP/crossmodal/images
Processing captions in english...
SBERT features ready. Timestamp: 2023-04-28 11:21:03.544696
CLIP features ready. Timestamp: 2023-04-28 11:21:03.828558
MCLIP features ready. Timestamp: 2023-04-28 11:21:28.767984
Encodings complete
----------------------------------------------------------------------
Processing captions in spanish...
SBERT features ready. Timestamp: 2023-04-28 11:21:29.418020
CLIP features ready. Timestamp: 2023-04-28 11:21:29.695009
MCLIP features ready. Timestamp: 2023-04-28 11:21:53.435306
Encodings complete
----------------------------------------------------------------------
Processing captions in italian...
SBERT features ready. Timestamp: 2023-04-28 11:21:54.176143
CLIP features ready. Timestamp: 2023-04-28 11:21:54.423764
MCLIP features ready. Timestamp: 2023-04-28 11:22:24.068257
Encodings complete
----------------------------------------------------------------------
Processing capti

In [30]:
show_plot(languages,sbert_MRR,clip_MRR,mclip_MRR)

NameError: name 'sbert_MRR' is not defined

In [17]:
show_plot(languages,s_acc,c_acc,m_acc,'Accuracy')

NameError: name 's_acc' is not defined

In [6]:
metrics_coco

NameError: name 'metrics_coco' is not defined